In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
train = pd.read_csv('../data/processed/processed_train.csv')

### Мы предлагаем обучить две модели, которые будут предсказывать два разных таргета. Во время обучения одной - другой будет являться признаком

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    ...
)